In [1]:
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, \
    DataCollection, bbox_to_dimensions, DownloadRequest, SentinelHubBatch
import os
import datetime
import matplotlib.pyplot as plt
from sentinelhub import SHConfig
from datetime import datetime
import math
import re, ast
import struct
import csv
import gdal
import fiona
import rasterio
import rasterio.mask
import swifter
from PIL import Image

In [19]:
# Test for import of this document
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

In [2]:
def cut_raster_with_shape (tiff, shape, outfile):
    with fiona.open(shape, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
        
    with rasterio.open(tiff) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True, nodata = -10000)
        out_meta = src.meta
        
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    with rasterio.open(outfile, "w", **out_meta) as dest:
        dest.write(out_image)

In [3]:
def raster_to_csv (filename):
    '''Takes a raster file (without the .tif ending) and returns the path to a csv written in the same directory
    containing the XYZ values from the raster file'''
    inDs = gdal.Open('{}.tif'.format(filename))
    outDS = gdal.Translate('{}.xyz'.format(filename), inDs, format='XYZ')
    os.rename('{}.xyz'.format(filename), '{}.csv'.format(filename))
    
    return (f'{filename}.csv')

In [4]:
def pre_process_csv(df,df_bool = True):
    '''This function takes a path to a csv file and returns a cleaned version limited only to the 
    land area of a country. (The CSV should be in the format Lon Lat Radiance as extraced by gdal toXYZ function)
    
    If df_bool is false you are passing a file to be read and preporcessed.'''

    if df_bool is False:
        print('here')
        df = pd.read_csv(path,header = None)
    
    df.columns = ['Lon','Lat','Radiance']
    df['Radiance'].replace(-10000.0, np.nan, inplace=True) 
    df = df.dropna()
    df['Radiance'] = df['Radiance'].apply(lambda row: max(0,row))
    df.reset_index(drop = True,inplace = True)
    
    return df


In [17]:
def is_outlier (row, df, n_sig = None):
    '''This function deals with outliers in the radiance values of a data frame with columns Lon, Lat, Radiance.
    A point is considered an outlier if it is greater than 100 or more than 3 standard deviations away from the 
    average radiance of a 1 km squared block surrounding the row in question'''
    
    if n_sig != None:
        thresh = df['Radiance'].std()*n_sig + df['Radiance'].mean()
        if row[-1] >= thresh:
            return False
    
    square = df[(df['Lon'] > row[0] - .05) & (df['Lon'] < row[0] + .05) & 
               (df['Lat'] > row[1] - .05) & (df['Lat'] < row[1] +.05)]
    mean = square.Radiance.mean()
    stdev = square.Radiance.std()
    
    # if the radiance is more 3 sigma away from the mean of its tile then it is an outlier
    if abs(row[-1] - mean) > stdev *3:
        return False

    # If not the above condition then the point is not an outlier and we keep it
    else:
        return True

In [7]:
def remove_outliers (df,n_sig = None):

    df['Drop_Outlier'] = df.swifter.apply(lambda row: is_outlier(row, df,n_sig), axis = 1)
    df_clean = df[df.Drop_Outlier]
    df_clean = df_clean.drop(['Drop_Outlier'],axis = 1)
    
    return df_clean

In [8]:
def round_down(number:float, decimals:int=1):
    """Returns a value rounded down to a specific number of decimal places (default is one decimal place)"""
    new_val = math.floor(number * (10**decimals)) / (10**decimals)
    
    return new_val

In [9]:
def agg_radiance (df, agg_size):
    '''This function takes a DataFrame with columns Lon, Lat, and Radiance (all of type float) and aggregates the 
    radiance column to regions of size agg_size in km squared (i.e. if you want to aggregate to 1 km squared enter
    1, 10 for 10km squared and so forth) - The parameter agg_size only takes integers of the form 10^x.'''
    
    size_dict = {1: 2, 10: 1, 100: 0, 1000: -1}
    new_df = df.sort_values('Lon',ascending = True)
    new_df['Lon'] = new_df['Lon'].apply(lambda num: round_down(num,size_dict[agg_size]))
    new_df['Lat'] = new_df['Lat'].apply(lambda num: round_down(num,size_dict[agg_size]))
    new_df = new_df.groupby(['Lon','Lat'],as_index=False).mean()
    
    return new_df

In [10]:
def make_boxes (row, step = .0999999):
    box = [row[0],row[1], row[0]+step, row[1] + step]
    return box

In [11]:
def get_boxes (df):
    '''This function takes a df with columns Lon, Lat, and Radiance aggregated to a factor of 10 and aggregates 
    these radiance labels to size 10 km squared (maximum size you can download from sentinel api at 10m level)'''
    
    df_new = df.sort_values('Lon',ascending = True)
    df_new['Lon'] = df_new['Lon'].apply(lambda num: round_down(num,1))
    df_new['Lat'] = df_new['Lat'].apply(lambda num: round_down(num,1))
    df_new = df_new.groupby(['Lon','Lat'],as_index=False).mean()
    df_new['boxes'] = df_new.apply(lambda row: make_boxes(row), axis = 1)
    
    return df_new

In [12]:
def get_sentinel_image (row, client_id, secret, res = 10):
    '''This function takes a list representing a latitude longitude box of approximately size 10km squared (this
    box should be a list of len 4) and a desired image resolution (10m, 20m or 60m with default 10m) in order to 
    return an array representing an image that corresponds to the lat lon box. '''
    
    config = SHConfig()
    config.sh_client_id = client_id
    config.sh_client_secret = secret
    
    evalscript_true_color = """
    //VERSION=3

        function setup() {
            return {
                input: [{
                    bands: ["B02", "B03", "B04"]
                }],
                output: {
                    bands: 3
                }
            };
        }

        function evaluatePixel(sample) {
            return [3.5*sample.B04, 3.5*sample.B03, 3.5*sample.B02];
        }
    """
    row_coords_wgs84 = row[-1]
    row_bbox = BBox(bbox=row_coords_wgs84, crs=CRS.WGS84)
    row_size = bbox_to_dimensions(row_bbox, resolution=res)
    if row_size[0] <= 2500 and row_size[1]<= 2500:
        request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L1C,
                    time_interval=('2019-01-01', '2019-12-31'),
                    mosaicking_order='leastCC'
                 )
             ],
            responses=[
                SentinelHubRequest.output_response('default', MimeType.PNG)
            ],
            bbox=row_bbox,
            size=row_size,
            config=config
        )
        return request_true_color.get_data()[0]
    else:
        print ('Region is too large to access, please limit image size and try again')
        return None

In [13]:
def map_sentinel_images (df, client_id, secret):
    '''Apply the get_sentinel_image function to an entire dataframe and return that dataframe (dataframe should
    be of columns Lon, Lat, Radiance, and Boxes)'''
    df['image'] = df.swifter.apply(lambda row: get_sentinel_image(row, client_id = client_id,
                                                    secret = secret), axis = 1)
    
    return df

In [1]:
def save_photos (df, directory, make = True):
    '''This function will save photos represented as ndArrays as jpeg photos in a specified directory. If make equals
    True a new directory will be made under the path specified under directory.'''
    
    if  not os.path.isdir(directory):
        os.mkdir(directory)
    for i in range(len(df)):
        lon = df['Lon'][i]
        lat = df['Lat'][i]
        rad = df['Radiance'][i]
        im_label = f'{lon}_{lat}_{rad}.jpeg'
        im = Image.fromarray(df['image'][i])
        full_label = directory +'/'+im_label
        im.save(directory +'/'+im_label)

In [14]:
def map_image_to_class (labels, images):
    '''This function takes two dataframes: Classes should be a dataframe with columns Lon Lat and Radiance while
    images should contain Lon, Lat, Radiance, boxes, and image. This function will map the large images contained
    in the images dataframe to each class label in classes contained within the geographical bounds of that larger
    image.'''
    labels = labels.copy()
    labels.index = labels.apply(lambda row: str(round_down(row[0], 1)) + " " + str(round_down(row[1], 1)), axis = 1)
    
    labels['image_large'] = np.nan
    
    image_dict = {}
    for i in range(len(images)):
        index = str(images['Lon'][i]) + " " + str(images['Lat'][i])
        key = images.index[i]
        labels['image_large'][index] = key
        image_dict[key] = images['image'][i]
        
    labels['image_large'] = labels['image_large'].map(image_dict, na_action='ignore') 
    
    labels.reset_index(level=0, inplace=True)
    
    return labels

In [15]:
def extract_image (row):
    '''This function takes a row and proportionally maps the radiance label to a specific portion of a larger image
    based on latitude and longitude'''
    img = row[4]
    img_shp = img.shape

    tile = row[0]
    tile_lon = float(tile.split(" ")[0])
    tile_lat = float(tile.split(" ")[1])

    lon = row[1]
    lat = row[2]

    lon_loc = round((lon - tile_lon) *10000)
    lat_loc = round((lat - tile_lat) *10000)

    lon_step = (.1*(img_shp[1]))
    lat_step = (.1*(img_shp[0]))
    
    lon_prop = (lon_loc//100)
    lat_prop = (lat_loc//100)

    lon_index_bottom = round(lon_step*lon_prop)
    lat_index_top = img_shp[0] - math.floor(lat_step*lat_prop)

    if lon_prop != 9:
        lon_index_top = round(lon_index_bottom + lon_step)
    else: 
        lon_index_top = round(lon_index_bottom + lon_step) +1
        
    lat_index_bottom = math.floor(lat_index_top - lat_step)


    if len(img_shp) == 3:
        mapped_img = img[lat_index_bottom: lat_index_top, lon_index_bottom: lon_index_top]
        return mapped_img
    else:
        return None
